###### Specifing run configurations and regularization constants. These are used to tweak the model.

In [1]:
# RUN CONFIGURATIONS
TEST = 1
CNN_ACT_F = "relu"
DNS_ACT_F = "sigmoid"
ARCH = f"CNN(64,128,256)_{CNN_ACT_F}_DENSE(128,64,32,12)_{DNS_ACT_F}"
IMG_SIZE = 60
N_SPLIT = 5
EPOCHS = 30
BATCH_SIZE = 250
VAL_SPLIT = 0.1


# REGULARIZATIONS
DROPOUT_RATE = 0.4
LAMBDA_1 = 1e-5
LAMBDA_2 = 1e-5

###### Loading the dataset from pickle

In [2]:
import pickle

X = pickle.load(open(f"X_{IMG_SIZE}.pickle", "rb"))
Y = pickle.load(open(f"Y_{IMG_SIZE}.pickle", "rb"))

###### Creating the report name for the corresponding tweak of the model

In [3]:
REPORT_NAME = f"{TEST} - {ARCH} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2"
REPORT_NAME

'3 - CNN(64,128,256)_relu_DENSE(128,64,32,12)_sigmoid - 60px_30e_250bs_10vs_REG_40do_10L1_10L2'

###### Importing tensorflow-gpu and other related libs and classes. Tensorboard is used to see live tables.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import time

C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Thilina\.conda\envs\tenso

In [5]:
MODEL_NAME = f"NN_GA_CNN_{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{MODEL_NAME}")

def create_cnn_model():
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), input_shape=(IMG_SIZE,IMG_SIZE,1)))  # input shape is IMG_SIZExIMG_SIZEx1
#     model.add(BatchNormalization(axis=3))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(128, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(256, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(128, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Dropout(rate=DROPOUT_RATE))
    
    model.add(Flatten())
    
#     model.add(Dense(128))
    model.add(Dense(128, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(12))
    model.add(Activation("softmax"))
    
    optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

###### Creating folder for saving reports of the model executions

In [6]:
import os

if not os.path.exists('reports'):
    os.makedirs('reports')

file = open(f"reports/{REPORT_NAME}.txt", "w")

###### Designed the 5-fold cross validation training and test with specific validation split. Final accuracy is averaged by the accuracies of the separate 5 folds. Time taken for the execution is also measured.

In [7]:
from sklearn.model_selection import KFold

# initializing variables for kFold run and average accuracy
current_fold = 0
sum_acc = 0
avg_acc = 0

start = time.process_time()

for train_index, test_index in KFold(N_SPLIT).split(X):
    current_fold += 1
    
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = create_cnn_model()
    
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VAL_SPLIT, callbacks=[tensorboard])
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(f"\nFOLD-{current_fold}: Loss={val_loss} , Accuracy={val_acc}\n")
    
    sum_acc += val_acc 
    file.write(f"{current_fold}-FOLD | Loss={round(val_loss,4)},\tAccuracy={round(val_acc,4)},\tAverage_Accuracy={round(sum_acc/current_fold,4)}\n")
    
    if(current_fold == N_SPLIT):
        avg_acc = round(sum_acc/current_fold,4)
    

avg_acc_line = f"\nAverage Accuracy : {round(avg_acc,4)}"

end = time.process_time()
time_taken = f"\nExecution Time\t : {round(end-start,4)}s"

print(avg_acc_line)
print(time_taken)

file.write(avg_acc_line)
file.write(time_taken)
    
file.close()

Instructions for updating:
Colocations handled automatically by placer.
Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 8s 2ms/sample - loss: 2.5572 - acc: 0.1041 - val_loss: 2.5444 - val_acc: 0.1395
Epoch 2/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5279 - acc: 0.1386 - val_loss: 2.5405 - val_acc: 0.1395
Epoch 3/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5245 - acc: 0.1386 - val_loss: 2.5378 - val_acc: 0.1395
Epoch 4/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5240 - acc: 0.1386 - val_loss: 2.5384 - val_acc: 0.1395
Epoch 5/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5239 - acc: 0.1386 - val_loss: 2.5377 - val_acc: 0.1395
Epoch 6/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5236 - acc: 0.1386 - val_loss: 2.5382 - val_acc: 0.1395
Epoch 7/30
3420/3420 [==============================] - 4s 1ms/sa

3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5273 - acc: 0.1377 - val_loss: 2.5375 - val_acc: 0.1395
Epoch 27/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5273 - acc: 0.1377 - val_loss: 2.5370 - val_acc: 0.1395
Epoch 28/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5272 - acc: 0.1377 - val_loss: 2.5374 - val_acc: 0.1395
Epoch 29/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5278 - acc: 0.1377 - val_loss: 2.5365 - val_acc: 0.1395
Epoch 30/30
950/950 [==============================] - 1s 622us/sample - loss: 2.5189 - acc: 0.1368

FOLD-2: Loss=2.5189123118551153 , Accuracy=0.13684210181236267

Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 5s 2ms/sample - loss: 2.6624 - acc: 0.1301 - val_loss: 2.5985 - val_acc: 0.1395
Epoch 2/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5542 - acc: 0.1348 - val_loss: 2.540

3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5240 - acc: 0.1395 - val_loss: 2.5382 - val_acc: 0.1395
Epoch 24/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5235 - acc: 0.1395 - val_loss: 2.5389 - val_acc: 0.1395
Epoch 25/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5235 - acc: 0.1395 - val_loss: 2.5383 - val_acc: 0.1395
Epoch 26/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5232 - acc: 0.1395 - val_loss: 2.5378 - val_acc: 0.1395
Epoch 27/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5234 - acc: 0.1395 - val_loss: 2.5381 - val_acc: 0.1395
Epoch 28/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5231 - acc: 0.1395 - val_loss: 2.5376 - val_acc: 0.1395
Epoch 29/30
3420/3420 [==============================] - 4s 1ms/sample - loss: 2.5232 - acc: 0.1395 - val_loss: 2.5383 - val_acc: 0.1395
Epoch 30/30
950/950 [==============================] 

###### Saving the model for later use

In [8]:
model.save(f"{REPORT_NAME}.model")

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 58, 58, 64)        640       
_________________________________________________________________
activation_28 (Activation)   (None, 58, 58, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 128)       73856     
_________________________________________________________________
activation_29 (Activation)   (None, 26, 26, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 10, 10, 256)       295168    
__________

###### To reset GPU memory after model training. But seems this doesn't work. Always need to restart the kernel and run.

In [10]:
tf.reset_default_graph()